In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run',
    port=80,
    username='mail_id@rapido.bike'
)

In [2]:
start_date = '2022-10-01'
end_date = '2022-10-31'

In [4]:
Link = f"""


WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(link_lifetime_last_ride_date) 
    link_lifetime_last_ride_date,
    link_lifetime_first_ride_date,
    --MAX(link_lifetime_rides) 
    link_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (link_lifetime_rides > 0 or link_lifetime_rides is not null or link_lifetime_last_ride_date is not null)
--GROUP BY 1,2
),

base_month_segment AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    customer_id,
    CASE
    WHEN link_lifetime_rides IS NULL OR DATE(month) = DATE(link_lifetime_first_ride_date) THEN 'LTR =0'
    WHEN link_lifetime_rides BETWEEN 1 AND 4 THEN 'LTR 1-4'
    WHEN (link_lifetime_rides BETWEEN 5 AND 20) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 5-20'
    WHEN (link_lifetime_rides BETWEEN 21 AND 50) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 21-50'
    WHEN (link_lifetime_rides > 50) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN 'LTR 50+'
    WHEN link_lifetime_rides >= 5 THEN 'LTR >=5'
    ELSE 'LTR =0'
    END ltr_link,
    CASE 
    WHEN link_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(link_lifetime_first_ride_date) THEN 'LTR=0'

    WHEN link_lifetime_rides BETWEEN 1 AND 4 and link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '0' DAY) THEN '1. 1_7_DAYS'
    WHEN link_lifetime_rides BETWEEN 1 AND 4 and link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '2. 8_14_DAYS'
    WHEN link_lifetime_rides BETWEEN 1 AND 4 and link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '21' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '3. 15_21_DAYS'
    WHEN link_lifetime_rides BETWEEN 1 AND 4 and link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '22' DAY) THEN '4. 22_30_DAYS'


    WHEN link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '5. LAST_30_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '45' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '6. 31_45_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '46' DAY) THEN '7. 46_60_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '8. 61_90_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '9. 91_180_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '99. 181_365_DAYS'
    WHEN link_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '999. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_link
FROM base_month_customer
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name = 'Link'
    AND customerid IS NOT NULL
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_link,'NEW') ltr_link,
        COALESCE(b.recency_link, 'NEW') recency_link,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_link,
        case 
        when ltr_link = 'LTR 1-4' then 1
        when ltr_link = 'LTR 5-20' then 2
        when ltr_link = 'LTR 21-50' then 3
        when ltr_link = 'LTR 50+' then 4
        when ltr_link = 'LTR >=5' then 5
        when ltr_link = 'LTR =0' then 6
        when ltr_link = 'NEW' then 7
        end row_order,
        recency_link,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_link,
        a.recency_link,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

b as 
(
SELECT
        month,
        8 row_order,
        'TOTAL' ltr_link,
        '' recency_link,
        SUM(existing_custr_count) existing_custr_count,
        100.00 existing_custr_distr,
        SUM(ao_users) ao_users,
        COALESCE(try(SUM(ao_users)*100.00/SUM(existing_custr_count)), 0) ao_conversion,
        SUM(fe_users) fe_users,
        COALESCE(try(SUM(fe_users)*100.00/SUM(existing_custr_count)), 0) fe_conversion,
        SUM(rr_users) rr_users,
        COALESCE(try(SUM(rr_users)*100.00/SUM(existing_custr_count)), 0) rr_conversion,
        COALESCE(try(SUM(rr_users)*100.00/SUM(fe_users)), 0) fe_rr,
        SUM(net_users) net_users,
        COALESCE(try(SUM(net_users)*100.00/SUM(existing_custr_count)), 0) net_conversion,
        COALESCE(try(SUM(net_users)*100.00/SUM(rr_users)), 0) rr_net,
        100.00 net_users_distr,
        SUM(net_rides) net_rides,
        100.00 net_rides_distr,
        COALESCE(try(SUM(net_rides)*1.0/SUM(net_users)), 0) rpc,
        SUM(discount) discount,
        SUM(subtotal) subtotal
FROM a
GROUP BY 1
),

final as 
(
select * from a
union
select * from b
)
select 
        month,
        ltr_link,recency_link,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,3

"""

In [5]:
df_link = pd.read_sql(Link, conn)

In [6]:
df_link

,month,ltr_link,recency_link,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-10-01,LTR 1-4,1. 1_7_DAYS,524792,1.86,400871,76.39,367659,70.06,268206,...,42.22,82.62,4.74,595353,4.16,2.69,2188464.0,36434535.0,6.006565,3.675910
1,2022-10-01,LTR 1-4,2. 8_14_DAYS,377473,1.34,243300,64.45,216216,57.28,135046,...,26.65,74.48,2.15,219172,1.53,2.18,866503.0,13784998.0,6.285841,3.953530
2,2022-10-01,LTR 1-4,3. 15_21_DAYS,334083,1.18,196231,58.74,171881,51.45,101667,...,22.04,72.44,1.58,147301,1.03,2.00,561528.0,9388136.0,5.981251,3.812113
3,2022-10-01,LTR 1-4,4. 22_30_DAYS,415149,1.47,221863,53.44,192785,46.44,108752,...,18.63,71.14,1.66,146378,1.02,1.89,574133.0,9407838.0,6.102709,3.922263
4,2022-10-01,LTR 1-4,6. 31_45_DAYS,611384,2.17,297217,48.61,255292,41.76,135047,...,15.36,69.53,2.01,169123,1.18,1.80,655235.0,11049072.0,5.930227,3.874310
5,2022-10-01,LTR 1-4,7. 46_60_DAYS,541037,1.92,237063,43.82,201117,37.17,103152,...,13.10,68.71,1.52,122842,0.86,1.73,492579.0,8155848.0,6.039580,4.009858
6,2022-10-01,LTR 1-4,8. 61_90_DAYS,956734,3.39,354475,37.05,296748,31.02,143731,...,10.06,66.94,2.06,163136,1.14,1.70,675897.0,10868965.0,6.218596,4.143151
7,2022-10-01,LTR 1-4,9. 91_180_DAYS,2303551,8.16,633867,27.52,527108,22.88,235515,...,6.67,65.28,3.29,252478,1.76,1.64,1096692.0,16920140.0,6.481578,4.343713
8,2022-10-01,LTR 1-4,99. 181_365_DAYS,2164498,7.67,358934,16.58,301305,13.92,131967,...,3.93,64.40,1.82,141003,0.99,1.66,661182.0,9525445.0,6.941219,4.689134
9,2022-10-01,LTR 1-4,999. 365_ABOVE,5492237,19.46,339452,6.18,287273,5.23,130760,...,1.55,65.26,1.83,149774,1.05,1.76,737190.0,9970166.0,7.393959,4.922016


## LTR = 0

In [12]:
ltr_0 = f"""

WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(link_lifetime_last_ride_date) 
    link_lifetime_last_ride_date,
    link_lifetime_first_ride_date,
    --MAX(link_lifetime_rides) 
    link_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month',cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (link_lifetime_rides > 0 or link_lifetime_rides is not null or link_lifetime_last_ride_date is not null)
--GROUP BY 1,2
),

ao_recent_ltr0 as
(
-- SELECT
--     customer_id,
--     max(day) ao_recency
-- FROM
    (
    SELECT 
        customerid customer_id,
        max(day) ao_recency
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month',cast('{start_date}' as date)) - INTERVAL '365' DAY, '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month',cast('{start_date}' as date)), '%Y-%m-%d') 
        -- AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IN (SELECT DISTINCT customer_id FROM base_month_customer)-- where link_lifetime_rides not between 1 and 10000)
        AND ao_sessions_unique_daily > 0
    GROUP BY 1
    )
-- GROUP BY 1
),

base_month_segment AS
(
SELECT * FROM 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    base_month_customer.customer_id,
    CASE
    WHEN link_lifetime_rides IS NULL OR DATE(month) = DATE(link_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN link_lifetime_rides BETWEEN 1 AND 4 THEN '1-4'
    WHEN (link_lifetime_rides BETWEEN 5 AND 20) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '5-20'
    WHEN (link_lifetime_rides BETWEEN 21 AND 50) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '21-50'
    WHEN (link_lifetime_rides > 50) AND (link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '50+'
    WHEN link_lifetime_rides >= 5 THEN '>=5'
    ELSE 'LTR=0'
    END ltr_link,
    
    CASE 
    WHEN link_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(link_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN (link_lifetime_rides BETWEEN 1 AND 4) AND link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) THEN '1. LAST_7_DAYS'
    WHEN (link_lifetime_rides BETWEEN 1 AND 4) AND link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '2. 8_14_DAYS'
    WHEN (link_lifetime_rides BETWEEN 1 AND 4) AND link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '21' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '3. 15_21_DAYS'
    WHEN (link_lifetime_rides BETWEEN 1 AND 4) AND link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '22' DAY) THEN '4. 22_30_DAYS'
    WHEN (link_lifetime_rides BETWEEN 1 AND 4) AND link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '45' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '5. 31_45_DAYS'
    WHEN (link_lifetime_rides BETWEEN 1 AND 4) AND link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '46' DAY) THEN '6. 46_60_DAYS'
    WHEN (link_lifetime_rides BETWEEN 1 AND 4) AND link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '7. 61_90_DAYS'    
    
    WHEN link_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '11. LAST_30_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '12. 31_90_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '8. 91_180_DAYS'
    WHEN link_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '9. 181_365_DAYS'
    WHEN link_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '10. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_link,
    
    CASE
    WHEN DATE(ao_recency) >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '7' DAY) THEN '1. LAST_7_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '14' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '8' DAY) THEN '2. 8_14_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '15' DAY) THEN '3. 15_30_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '60' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '4. 31_60_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '61' DAY) THEN '5. 61_90_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '6. 91_180_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '7. 181_365_DAYS'
    WHEN DATE(ao_recency) BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '730' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '366' DAY) THEN '8.1_2_YEARS'
    ELSE 'NO_AO'
    END ao_recency
FROM base_month_customer 
LEFT JOIN ao_recent_ltr0 b on base_month_customer.customer_id = b.customer_id
-- where link_lifetime_rides not between 1 and 10000
)
WHERE ltr_link = 'LTR=0' AND recency_link = 'LTR=0'
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month',cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name = 'Link'
    AND customerid IS NOT NULL
    AND customerid IN (SELECT DISTINCT customer_id FROM base_month_customer)-- where link_lifetime_rides not between 1 and 10000)
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month',cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month',cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
        AND customerid IN (SELECT DISTINCT customer_id FROM base_month_customer)-- where link_lifetime_rides not between 1 and 10000)
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_link,'NEW') ltr_link,
        COALESCE(b.recency_link, 'NEW') recency_link,
        ao_recency,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_link,
        case 
        when ltr_link = '1-4' then 1
        when ltr_link = '5-20' then 2
        when ltr_link = '21-50' then 3
        when ltr_link = '50+' then 4
        when ltr_link = '>=5' then 5
        when ltr_link = 'LTR=0' then 6
        when ltr_link = 'NEW' then 7
        end row_order,
        recency_link,
        ao_recency,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4,5
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_link,
        a.recency_link,
        ao_recency,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

final as 
(
select * from a
)
select 
        month,
        ltr_link,recency_link,ao_recency,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,4

"""

In [13]:
df_ltr_0 = pd.read_sql(ltr_0, conn)

In [14]:
df_ltr_0

,month,ltr_link,recency_link,ao_recency,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-10-01,LTR=0,LTR=0,1. LAST_7_DAYS,561611,1.99,454468,80.92,413115,73.56,...,17.92,54.36,2.63,210982,1.65,2.10,1440787.0,14047140.0,10.256800,6.828957
1,2022-10-01,LTR=0,LTR=0,2. 8_14_DAYS,221464,0.78,121945,55.06,101341,45.76,...,8.48,53.89,0.49,32422,0.25,1.73,272354.0,2236004.0,12.180390,8.400284
2,2022-10-01,LTR=0,LTR=0,3. 15_30_DAYS,347103,1.23,143659,41.39,115018,33.14,...,6.30,55.72,0.57,35994,0.28,1.65,307835.0,2515751.0,12.236306,8.552398
3,2022-10-01,LTR=0,LTR=0,4. 31_60_DAYS,420895,1.49,106302,25.26,82906,19.70,...,4.31,58.37,0.47,30247,0.24,1.67,260567.0,2138307.0,12.185668,8.614639
4,2022-10-01,LTR=0,LTR=0,5. 61_90_DAYS,293156,1.04,43126,14.71,34085,11.63,...,2.97,61.51,0.23,14462,0.11,1.66,127482.0,999077.0,12.759977,8.814963
5,2022-10-01,LTR=0,LTR=0,6. 91_180_DAYS,666773,2.36,55825,8.37,47201,7.08,...,2.06,62.28,0.36,23376,0.18,1.70,206861.0,1666794.0,12.410712,8.849290
6,2022-10-01,LTR=0,LTR=0,7. 181_365_DAYS,646882,2.29,30605,4.73,27164,4.20,...,1.44,64.97,0.24,16437,0.13,1.77,143254.0,1146898.0,12.490561,8.715337
7,2022-10-01,LTR=0,LTR=0,NO_AO,1518974,5.38,37101,2.44,33391,2.20,...,0.87,69.52,0.35,24612,0.19,1.86,204206.0,1664034.0,12.271744,8.297010
8,2022-10-01,NEW,NEW,None,0,0.00,7794462,0.00,6987722,0.00,...,0.00,79.24,94.66,12370337,96.95,3.41,40505908.0,725081068.0,5.586397,3.274439
